# Rare Variant Association Analysis with RVtests (ADNI)

This notebook provides a full workflow for rare variant burden testing in the ADNI dataset using PLINK data with RVtests.

## Workflow Overview

### 1. Data Preparation & QC
- Perform PLINK-based QC for rare (MAF 0.005–0.01) and common variants
- LD pruning and PCA generation for population structure control
- Generate phenotype, covariate, and keep files for AD vs CN baseline samples

### 2. Dataset Conversion
- Filter rare variant dataset to matched samples
- Convert PLINK binary files to VCF, followed by compression and indexing

### 3. Association Testing (RVtests)
- Gene-based rare variant testing using:
  - CMC (burden test)
  - SKAT-O (kernel test)
  - Variable Threshold (Price)
- Adjusts for covariates (age, sex, education, APOE4, PCs)

### 4. Post-Analysis
- Aggregate and compare results across tests
- Identify genes significant in multiple methods
- Export summary statistics and significant findings

### 5. Variant Extraction
- Extract significant loci from VCF using bcftools
- Apply allele frequency filters for rare variant validation
- Generate RSID lists for candidate genes


---

### Citations

> Petersen RC, et al. Alzheimer's Disease Neuroimaging Initiative. Alzheimers Dement. 2010 May;6(3):238-46. </p>
> Xiaowei Zhan, et al. RVTESTS: An Efficient and Comprehensive Tool for Rare Variant Association Analysis Using Sequence Data. Bioinformatics. 2016 32: 1423-1426.

### 01 Data prepartion & QC
i. PLINK QC for rare variants <br>
ii. PLINK QC for common variants <br>
iii. LD pruning <br>
iv. PCA <br>
v. Pheno & covar file creation <br>

In [ ]:
# 01_data_prep_qc
# i. PLINK QC for rare variants


##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/inputs/ADNI_merged"
OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/rare_plink/ADNI2_rare"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################


# ── QC rare variants ─────────────────────────────

plink --bfile "${BFILE}" \
      --maf 0.005 \
      --max-maf 0.01 \
      --geno 0.05 \
      --mind 0.05 \
      --hwe 1e-6 \
      --mac 1 \
      --not-chr 0,M \
      --output-chr M \
      --make-bed \
      --out "${OUTPUT}"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}.[bed|bim|fam]"
echo "🏁 QC (rare) complete."

In [ ]:
# 01_data_prep_qc
# ii. PLINK QC for common variants


##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/inputs/ADNI_merged"
OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/common_plink/ADNI2_common"


##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################


# ── QC common variants ─────────────────────────────

plink --bfile "$BFILE" \
      --maf 0.01 \
      --geno 0.05 \
      --mind 0.05 \
      --hwe 1e-6 \
      --mac 1 \
      --not-chr 0,M \
      --output-chr M \
      --make-bed \
      --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}.[bed|bim|fam]"
echo "🏁 QC (common) complete."

In [ ]:
# 01_data_prep_qc
# iii. LD pruning

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/common_plink/ADNI2_common"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/LD_pruning/ADNI2_pruned"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################


# ── LD pruning ─────────────────────────────

plink \
  --bfile "$BFILE" \
  --indep-pairwise 200 50 0.3 \
  --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}"
echo "🏁 LD pruning complete.

In [ ]:
# 01_data_prep_qc
# iv. PCA generation on common variants set

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/common_plink/ADNI2_common"

PRUNED="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/LD_pruning/ADNI2_pruned.prune.in"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/PCA/ADNI2_PCA"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

# ── PCA generation ─────────────────────────────

plink \
  --bfile "$BFILE" \
  --extract "$PRUNED" \
  --pca 10 header \
  --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}"
echo "🏁 PCA complete."


In [ ]:
# 01_data_prep_qc
# v. Generate pheno and covar files

from pathlib import Path
import pandas as pd
import os

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

ADNI_MERGE_PATH = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\01_data_prep_qc\inputs\ADNIMERGE_15May2025.csv"
PLINK_FAM_PATH  = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\01_data_prep_qc\outputs\rare_plink\ADNI2_rare.fam"

PCA_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\01_data_prep_qc\outputs\PCA\ADNI2_PCA.eigenvec"

OUTPUT_DIR = Path(r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\01_data_prep_qc\outputs\pheno_covar")

NUM_PCS = 10  # Number of principal components to merge

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

# ── Load data  ─────────────────────────────

def load_pca_data(pca_path, num_pcs):
    print(f"📖 Loading PCA data with {num_pcs} components...")
    pca_columns = ["fid", "iid"] + [f"PC{i}" for i in range(1, num_pcs + 1)]
    pca = pd.read_csv(pca_path, sep=r'\s+', header=0, names=pca_columns, dtype={"fid": str, "iid": str})
    print(f"✅ Loaded PCA data for {len(pca)} samples")
    return pca

def save_keep_list(pheno_df, outdir):
    keep_path = outdir / "ADNI_AD_vs_CN.keep"
    pheno_df[['fid', 'iid']].to_csv(keep_path, sep='\t', index=False, header=False)
    print(f"📄 Keep-list saved: {keep_path}")
    return keep_path

def main():
    print("📥 Loading ADNI phenotype and PLINK sample data...")
    adni_data = pd.read_csv(ADNI_MERGE_PATH, low_memory=False)
    plink_fam = pd.read_csv(
        PLINK_FAM_PATH, sep=r'\s+', header=None,
        names=['fid', 'iid', 'PAT', 'MAT', 'SEX', 'PHENOTYPE']
    )

    # ── Filter AD vs CN ─────────────────────────────
    ad_cn_data = adni_data[(adni_data['VISCODE'] == 'bl') & (adni_data['DX_bl'].isin(['AD', 'CN']))].copy()
    print(f"🔍 Baseline AD vs CN: {len(ad_cn_data)} samples (before matching)")

    # ── Match PTIS and IIDM map FID ─────────────────────────────
    ad_cn_data['iid'] = ad_cn_data['PTID'].astype(str)
    ad_cn_data = ad_cn_data[ad_cn_data['iid'].isin(plink_fam['iid'].astype(str))]
    iid_to_fid = dict(zip(plink_fam['iid'].astype(str), plink_fam['fid']))
    ad_cn_data['fid'] = ad_cn_data['iid'].map(iid_to_fid)

    if ad_cn_data['fid'].isnull().any():
        missing = ad_cn_data[ad_cn_data['fid'].isnull()]['iid'].tolist()
        print("⚠️ WARNING: Some samples could not be mapped to FIDs")
        print(f"Missing FID for: {missing}")

    print(f"✅ After genetic match: {len(ad_cn_data)} samples => "
          f"{(ad_cn_data['DX_bl']=='AD').sum()} AD, {(ad_cn_data['DX_bl']=='CN').sum()} CN")

    # ── create phenotypes and covariates ─────────────────────────────
    ad_cn_data['sex'] = ad_cn_data['PTGENDER'].map({'Male': 1, 'Female': 0})
    ad_cn_data['age'] = ad_cn_data['AGE']
    ad_cn_data['education'] = ad_cn_data['PTEDUCAT']
    ad_cn_data['apoe4_count'] = ad_cn_data['APOE4']

    phenotype_cols = ['fid', 'iid', 'DX_bl', 'age', 'sex', 'education', 'apoe4_count', 'PTID']
    pheno_df = ad_cn_data[phenotype_cols].dropna(subset=['DX_bl', 'age', 'sex', 'education', 'apoe4_count'])
    print(f"🔍 Final sample size: {len(pheno_df)} "
          f"({(pheno_df['DX_bl']=='AD').sum()} AD, {(pheno_df['DX_bl']=='CN').sum()} CN)")

    # ── merge with parentals IDs and numeric sex from FAM ─────────────────────────────
    fam_cols = plink_fam[['fid', 'iid', 'PAT', 'MAT', 'SEX']].copy()
    fam_cols.rename(columns={'PAT': 'fatid', 'MAT': 'matid', 'SEX': 'sex_fam'}, inplace=True)


# ── PHENOTYPE FILE GENERATION (y1 = AD vs CN) ─────────────────────────────

    pheno_merged = pd.merge(pheno_df, fam_cols, on=['fid', 'iid'], how='left')
    pheno_merged['DX_bl'] = pheno_merged['DX_bl'].str.strip().str.upper()
    pheno_merged['y1'] = pheno_merged['DX_bl'].map({'CN': 1, 'AD': 2}).astype('Int64')

    pheno_final = pheno_merged[['fid', 'iid', 'fatid', 'matid', 'sex_fam', 'y1']].copy()
    pheno_final.rename(columns={'sex_fam': 'sex'}, inplace=True)
    pheno_final.to_csv(
        OUTPUT_DIR / "ADNI_AD_vs_CN.pheno",
        sep='\t', index=False, na_rep='NA'
    )


# ── COVARIATE FILE GENERATION (covars + PCs, no duplicate sex) ─────────────────────────────

    covar_base = pheno_df[['fid', 'iid', 'age', 'education', 'apoe4_count']].copy()
    covar_merged = pd.merge(covar_base, fam_cols, on=['fid', 'iid'], how='left')

    # Load PCA data
    pca_df = load_pca_data(PCA_FILE, NUM_PCS)

    # Merge covariates + PCA
    covar_with_pcs = pd.merge(covar_merged, pca_df.drop(columns='fid'), on='iid', how='left')

    # Reorder: fid iid fatid matid sex age education apoe4_count PC1...PCn
    covar_final = covar_with_pcs[['fid', 'iid', 'fatid', 'matid', 'sex_fam',
                                  'age', 'education', 'apoe4_count'] +
                                 [f"PC{i}" for i in range(1, NUM_PCS + 1)]].copy()
    covar_final.rename(columns={'sex_fam': 'sex'}, inplace=True)

    covar_final.to_csv(
        OUTPUT_DIR / "ADNI_AD_vs_CN.covar",
        sep='\t', index=False, na_rep='NA'
    )

    # Save keep-list file for PLINK
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    keep_path = save_keep_list(pheno_df, OUTPUT_DIR)

    # Save summary and stats
    pheno_df.to_csv(
        OUTPUT_DIR / "ADNI_AD_vs_CN_summary.txt",
        sep='\t', index=False, na_rep='NA'
    )

    summary_stats = pheno_df.groupby('DX_bl').agg({
        'age': ['count', 'mean', 'std'],
        'sex': 'mean',
        'education': ['mean', 'std'],
        'apoe4_count': ['mean', 'std']
    }).round(2)

    summary_stats.to_csv(
        OUTPUT_DIR / "ADNI_AD_vs_CN_summary_stats.txt",
        sep='\t'
    )

    print(f"\n✅ Files saved to: {OUTPUT_DIR}")
    print(f"✅ Phenotype file: {OUTPUT_DIR / 'ADNI_AD_vs_CN.pheno'}")
    print(f"✅ Covariate file: {OUTPUT_DIR / 'ADNI_AD_vs_CN.covar'}")
    print(f"🏁 Processing complete.")

if __name__ == "__main__":
    main()

### 02 Dataset conversion
i. Filter rare variant dataset to matched samples <br>
ii. Convert to VCF (compression and indexing) <br>
iii. Annotation (create annotated vcf to filer by variants)

In [ ]:
# 02_dataset_conversion
# i. filter plink rare to keep only samples with pheno data


##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

BFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/rare_plink/ADNI2_rare"
OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/ADNI2_AD_CN"
KEEP="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/pheno_covar/ADNI_AD_vs_CN.keep"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################


# ── QC rare variants ─────────────────────────────

plink --bfile "$BFILE" \
      --keep "$KEEP" \
      --make-bed \
      --out "$OUTPUT"
STATUS=$?

if [ $STATUS -ne 0 ]; then
  echo "❌ ERROR: PLINK QC failed with exit code $STATUS"
  exit $STATUS
fi

echo "✅ Files saved: ${OUTPUT}.[bed|bim|fam]"
echo "🏁 complete."

In [ ]:
# 02_dataset_conversion
# ii. convert to compressed vcf + index

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

INPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/ADNI2_AD_CN"

OUTPUT="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/ADNI2_AD_CN_converted"

VCF="${OUTPUT}.vcf"

BGZ="${VCF}.gz"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

# ── convert to vcf & compress + index ─────────────────────────────

plink \
    --bfile "${INPUT}" \
    --recode vcf-iid \
    --out "${OUTPUT}"

bgzip -f "${VCF}"
tabix -p vcf "${BGZ}"

echo "✅ Files saved: ${BGZ} and ${BGZ}.tbi"
echo "🏁 VCF conversion, compression, and indexing complete"


In [ ]:
# 02_dataset_conversion [optional]
# iii. produce annotated vcf file to filter variants on expression

cd anno

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

INPUT_VCF="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/ADNI2_AD_CN_converted.vcf.gz"
OUTPUT_VCF="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/annotation/ADNI2_AD_CN_anno.vcf.gz"
REFERENCE_FA="/home/swmitchell/anno/resources/hs37d5.fa"
GENE_ANNOTATION="/home/swmitchell/anno/resources/refFlat_hg19.txt.gz"
PRIORITY_FILE="/home/swmitchell/anno/resources/priority.txt"
CODON_FILE="/home/swmitchell/anno/resources/codon.txt"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

./executable/anno \
  -i "$INPUT_VCF" \
  -o "$OUTPUT_VCF" \
  -r "$REFERENCE_FA" \
  -g "$GENE_ANNOTATION" \
  -p "$PRIORITY_FILE" \
  -c "$CODON_FILE" \
  --indexOutput

### 03 Association Testing (RVtests)
i. Gene-based rare variant testing using:
  - CMC (burden test)
  - SKAT-O (kernel test)
  - Variable Threshold (Price)

In [ ]:
#rvtest

#groupwise tests
# example

rvtest --inVcf rarevariants.vcf.gz \
   --pheno pheno.ped \
   --out output \
   --geneFile refFlat_hg19.txt.gz \ # to specify the gene range in a refFlat format
   --burden cmc \ # collapsing and combine rare variants
   --vt price \
   --kernel skato \
   --covar example.covar \
   --covar-name age,sex,education,apoe4_count,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
   --freqUpper 0.01 \ # Specify upper minor allele frequency bound to be included in analysis
   --freqLower 0.001 \ # Specify lower minor allele frequency bound to be included in analysis


In [ ]:
# 03_rvtests
# i. run Rvtests

cd rvtests

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

# use normal vcf for non-annotation run
INVCF="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/ADNI2_AD_CN_converted.vcf.gz"

# if filtering based on annotation, need annotated vcf
#INVCF="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/annotation/ADNI2_AD_CN_anno.vcf.gz"

PHENO="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/pheno_covar/ADNI_AD_vs_CN.pheno"

COVAR="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/01_data_prep_qc/outputs/pheno_covar/ADNI_AD_vs_CN.covar"

GENEFILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/03_rvtests/inputs/refFlat.txt.gz"

OUTDIR="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/03_rvtests/outputs"

OUTFILE="${OUTDIR}/ADNI2_rvtest"

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

# ── run rvtests ─────────────────────────────

./executable/rvtest \
  --inVcf "$INVCF" \
  --pheno "$PHENO" \
  --out "$OUTFILE" \
  --geneFile "$GENEFILE" \
  --burden cmc \
  --vt price \
  --kernel skato \
  --covar "$COVAR" \
  --covar-name age,sex,education,apoe4_count,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
  --freqUpper 0.01 \
  --freqLower 0.005 \
  --noweb \
 # --annoType Start_Gain|Stop_Loss|Start_Loss|Essential_Splice_Site|Stop_Gain|Normal_Splice_Site|Synonymous|Nonsynonymous \ #filter variants based on annotation
  --outputRaw



### 04 Post-Analysis
- Aggregate and compare results across tests
- Identify genes significant in multiple methods
- Export summary statistics and significant findings


In [ ]:
# 04_post_analysis
# Processes CMC, SKAT-O, and Variable Threshold Price test results,
# Applies multiple testing corrections and identifies significant genes for further investigation.

from pathlib import Path
import pandas as pd
import numpy as np
import sys
from scipy import stats
import statsmodels.stats.multitest as smm

##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

# Input file paths
CMC_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\03_rvtests\outputs\ADNI2_rvtest.CMC.assoc"
SKATO_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\03_rvtests\outputs\ADNI2_rvtest.SkatO.assoc"
VT_FILE = r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\03_rvtests\outputs\ADNI2_rvtest.VariableThresholdPrice.assoc"

# Output directory
OUTPUT_DIR = Path(r"C:\Users\B00731414\OneDrive - Ulster University\6. Code\ADNI\5. Rvtests\04_post_analysis")

# Significance thresholds
HIGHLY_SIGNIFICANT_THRESHOLD = 0.001
SIGNIFICANT_THRESHOLD = 0.01
SUGGESTIVE_THRESHOLD = 0.05
PRIMARY_THRESHOLD = 0.05  # Primary reporting threshold

# Display settings
TOP_RESULTS_TO_SHOW = 20
MAX_GENES_TO_LIST = 10

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################


# ── Multiple testing corrections ─────────────────────────────

# Apply multiple testing corrections to p-values.
# return df with add cols for corrected p-values

def apply_multiple_testing_correction(df, p_column='P_value', methods=['bonferroni', 'fdr_bh']):
    df = df.copy()
    valid_pvals = df[p_column].notna() & (df[p_column] > 0)
    for method in methods:
        if method == 'bonferroni':
            # Bonferroni correction
            n_tests = valid_pvals.sum()
            df['p_bonferroni'] = df[p_column] * n_tests
            df['p_bonferroni'] = df['p_bonferroni'].clip(upper=1.0)  # Cap at 1.0
            df['sig_bonferroni'] = df['p_bonferroni'] < 0.05
        elif method == 'fdr_bh':
            # Benjamini-Hochberg FDR correction
            pvals = df[p_column][valid_pvals].values
            rejected, p_corrected, _, _ = smm.multipletests(pvals, method='fdr_bh', alpha=0.05)
            # Create full arrays with NaN for missing values
            df['p_fdr_bh'] = np.nan
            df['sig_fdr_bh'] = False
            df.loc[valid_pvals, 'p_fdr_bh'] = p_corrected
            df.loc[valid_pvals, 'sig_fdr_bh'] = rejected
    return df

# calculate sig thresholds for Bonf / FDR
def calculate_significance_thresholds(n_tests):
    bonferroni_threshold = 0.05 / n_tests
    print(f"\nSignificance Thresholds:")
    print(f"Nominal p < 0.05")
    print(f"Bonferroni p < {bonferroni_threshold:.2e} (0.05/{n_tests})")
    print(f"FDR q < 0.05 (adaptive threshold)")
    return {
        'nominal': 0.05,
        'bonferroni': bonferroni_threshold,
        'fdr': 0.05  # q-value threshold
    }

# ── Correction reporting ─────────────────────────────
def generate_correction_summary_report(best_results_corrected, output_dir):
    summary_stats = {
        'Total_Genes': len(best_results_corrected),
        'Nominal_Significant': (best_results_corrected['P_value'] < 0.05).sum(),
        'Bonferroni_Significant': (best_results_corrected['p_bonferroni'] < 0.05).sum(),
        'FDR_Significant': (best_results_corrected['sig_fdr_bh']).sum(),
        'Bonferroni_Threshold': f"{0.05/len(best_results_corrected):.2e}",
    }
    # Save summary
    summary_df = pd.DataFrame([summary_stats])
    summary_df.to_csv(output_dir / "multiple_testing_summary.csv", index=False)
    # save lists of significant genes by different criteria
    if summary_stats['FDR_Significant'] > 0:
        fdr_genes = best_results_corrected[best_results_corrected['sig_fdr_bh']]['Gene'].tolist()
        pd.DataFrame({'Gene': fdr_genes}).to_csv(output_dir / "FDR_significant_genes.csv", index=False)
    if summary_stats['Bonferroni_Significant'] > 0:
        bonf_genes = best_results_corrected[best_results_corrected['sig_bonferroni']]['Gene'].tolist()
        pd.DataFrame({'Gene': bonf_genes}).to_csv(output_dir / "Bonferroni_significant_genes.csv", index=False)
    return summary_stats

# ── Load and process Rvtests results ─────────────────────────────

def load_and_process_files(cmc_file, skato_file, vt_file):
    results = {}
    try:
        cmc_df = pd.read_csv(cmc_file, sep='\t')
        cmc_df['Test_Type'] = 'CMC'
        cmc_df['P_value'] = cmc_df['Pvalue']
        results['CMC'] = cmc_df
        print(f"📂 Loaded CMC file: {len(cmc_df)} genes")
    except Exception as e:
        print(f"❌ Error loading CMC file: {e}")
        return None
    try:
        skato_df = pd.read_csv(skato_file, sep='\t')
        skato_df['Test_Type'] = 'SKAT-O'
        skato_df['P_value'] = skato_df['Pvalue']
        results['SKAT-O'] = skato_df
        print(f"📂 Loaded SKAT-O file: {len(skato_df)} genes")
    except Exception as e:
        print(f"❌ Error loading SKAT-O file: {e}")
        return None
    try:
        vt_df = pd.read_csv(vt_file, sep='\t')
        vt_df['Test_Type'] = 'Variable_Threshold'
        vt_df['P_value'] = vt_df['PermPvalue']
        results['Variable_Threshold'] = vt_df
        print(f"📂 Loaded Variable Threshold file: {len(vt_df)} genes")
    except Exception as e:
        print(f"❌ Error loading Variable Threshold file: {e}")
        return None
    return results

# ── Significance metrics ─────────────────────────────
# Calculate -log10(p) values and assign defined sig categories

def calculate_significance_metrics(df, p_col='P_value'):
    df = df.copy()
    df['neg_log10_p'] = -np.log10(df[p_col].replace(0, 1e-300))  # handle p=0
    df['significance_level'] = pd.cut(
        df[p_col],
        bins=[0, HIGHLY_SIGNIFICANT_THRESHOLD, SIGNIFICANT_THRESHOLD, SUGGESTIVE_THRESHOLD, 1.0],
        labels=['Highly_Significant', 'Significant', 'Suggestive', 'Not_Significant'],
        include_lowest=True
    )
    return df

# ── Merge results ─────────────────────────────
def merge_results(results_dict):
    merged_results = []
    for test_type, df in results_dict.items():
        if test_type == 'CMC':
            cols = ['Gene', 'RANGE', 'N_INFORMATIVE', 'NumVar', 'NumPolyVar',
                    'NonRefSite', 'P_value', 'Test_Type', 'neg_log10_p', 'significance_level']
        elif test_type == 'SKAT-O':
            cols = ['Gene', 'RANGE', 'N_INFORMATIVE', 'NumVar', 'NumPolyVar',
                    'Q', 'rho', 'P_value', 'Test_Type', 'neg_log10_p', 'significance_level']
        else:  # Variable Threshold
            cols = ['Gene', 'RANGE', 'N_INFORMATIVE', 'NumVar', 'NumPolyVar',
                    'OptFreq', 'Zmax', 'Stat', 'P_value', 'Test_Type', 'neg_log10_p', 'significance_level']
        available_cols = [col for col in cols if col in df.columns]
        merged_results.append(df[available_cols])
    combined_df = pd.concat(merged_results, ignore_index=True, sort=False)
    return combined_df

# ── Gene-level aggregation ─────────────────────────────
# select best result per gene
def find_best_result_per_gene(combined_df):
    best_results = combined_df.loc[combined_df.groupby('Gene')['P_value'].idxmin()].copy()
    return best_results.sort_values('P_value')

# ── Cross-test significance ─────────────────────────────
# get cross-test sig for each gene
def analyze_cross_test_significance(combined_df, threshold=PRIMARY_THRESHOLD):
    gene_test_matrix = combined_df.pivot(index='Gene', columns='Test_Type', values='P_value')
    significant_counts = (gene_test_matrix < threshold).sum(axis=1)
    all_tests_sig = significant_counts[significant_counts == 3].index.tolist()
    two_tests_sig = significant_counts[significant_counts == 2].index.tolist()
    one_test_sig = significant_counts[significant_counts == 1].index.tolist()
    cross_test_results = []
    for gene in combined_df['Gene'].unique():
        gene_data = combined_df[combined_df['Gene'] == gene]
        result = {'Gene': gene}
        if 'RANGE' in gene_data: result['RANGE'] = gene_data['RANGE'].iloc[0]
        if 'N_INFORMATIVE' in gene_data: result['N_INFORMATIVE'] = gene_data['N_INFORMATIVE'].iloc[0]
        if 'NumVar' in gene_data: result['NumVar'] = gene_data['NumVar'].iloc[0]
        if 'NumPolyVar' in gene_data: result['NumPolyVar'] = gene_data['NumPolyVar'].iloc[0]
        for test_type in ['CMC', 'SKAT-O', 'Variable_Threshold']:
            test_data = gene_data[gene_data['Test_Type'] == test_type]
            if len(test_data) > 0:
                p_val = test_data['P_value'].iloc[0]
                result[f'{test_type}_pval'] = p_val
                result[f'{test_type}_sig'] = 'Yes' if p_val < threshold else 'No'
            else:
                result[f'{test_type}_pval'] = np.nan
                result[f'{test_type}_sig'] = 'N/A'
        result['Tests_Significant'] = sum([1 for test in ['CMC', 'SKAT-O', 'Variable_Threshold']
                                           if result[f'{test}_sig'] == 'Yes'])
        result['Min_Pvalue'] = min([result[f'{test}_pval']
                                    for test in ['CMC', 'SKAT-O', 'Variable_Threshold']
                                    if not pd.isna(result[f'{test}_pval'])])
        cross_test_results.append(result)
    cross_test_df = pd.DataFrame(cross_test_results)
    cross_test_df = cross_test_df.sort_values(['Tests_Significant', 'Min_Pvalue'], ascending=[False, True])
    return cross_test_df, all_tests_sig, two_tests_sig, one_test_sig

# ── Reporting ─────────────────────────────
def generate_summary_report(results_dict, combined_df, best_results, cross_test_df,
                            all_tests_sig, two_tests_sig, one_test_sig):
    print("\n" + "="*80)
    print("RARE VARIANT TESTING RESULTS SUMMARY")
    print("="*80)
    print(f"Significance thresholds:")
    print(f"  - Highly significant: p < {HIGHLY_SIGNIFICANT_THRESHOLD}")
    print(f"  - Significant: p < {SIGNIFICANT_THRESHOLD}")
    print(f"  - Suggestive: p < {SUGGESTIVE_THRESHOLD}")
    print(f"  - Primary threshold: p < {PRIMARY_THRESHOLD}")
    print(f"\n🔍 Overall statistics:")
    print(f"Total unique genes analysed: {len(combined_df['Gene'].unique())}")
    print(f"Total test results processed: {len(combined_df)}")
    print(f"\n🧪 Results by test type:")
    for test_type in ['CMC', 'SKAT-O', 'Variable_Threshold']:
        test_df = combined_df[combined_df['Test_Type'] == test_type]
        sig_count = len(test_df[test_df['P_value'] < PRIMARY_THRESHOLD])
        print(f" - {test_type}: {len(test_df)} genes, {sig_count} significant")
    print(f"\n📊 Cross-test significance analysis:")
    print(f"Genes significant in all 3 tests: {len(all_tests_sig)}")
    print(f"Genes significant in 2 tests: {len(two_tests_sig)}")
    print(f"Genes significant in 1 test: {len(one_test_sig)}")
    print(f"\n📌 Significance summary (best result per gene):")
    sig_summary = best_results['significance_level'].value_counts()
    for level in ['Highly_Significant', 'Significant', 'Suggestive', 'Not_Significant']:
        print(f" - {level}: {sig_summary.get(level, 0)} genes")
    print(f"\n🏆 Top {TOP_RESULTS_TO_SHOW} results:")
    display_cols = ['Gene', 'Test_Type', 'P_value', 'neg_log10_p', 'NumVar', 'NumPolyVar']
    sig_results = best_results[best_results['P_value'] < PRIMARY_THRESHOLD]
    if len(sig_results) > 0:
        print(sig_results[display_cols].head(TOP_RESULTS_TO_SHOW).to_string(index=False, float_format='%.2e'))
    else:
        print("No significant results found.")
        print(best_results[display_cols].head(TOP_RESULTS_TO_SHOW).to_string(index=False, float_format='%.2e'))

# ── Save outputs to csv ─────────────────────────────

def save_results(combined_df, best_results, cross_test_df, output_dir):
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True, parents=True)
    combined_df.to_csv(output_path / "all_rare_variant_results.csv", index=False)
    best_results.to_csv(output_path / "best_results_per_gene.csv", index=False)
    cross_test_df.to_csv(output_path / "cross_test_significance_analysis.csv", index=False)
    significant_results = best_results[best_results['P_value'] < PRIMARY_THRESHOLD]
    if len(significant_results) > 0:
        significant_results.to_csv(output_path / "significant_results.csv", index=False)
    multi_test_sig = cross_test_df[cross_test_df['Tests_Significant'] >= 2]
    if len(multi_test_sig) > 0:
        multi_test_sig.to_csv(output_path / "multi_test_significant_genes.csv", index=False)
    print(f"\n✅ Results saved to: {output_dir}")
    return output_path

# ── Main ─────────────────────────────
def main():
    # Check files exist
    for file_path in [CMC_FILE, SKATO_FILE, VT_FILE]:
        if not Path(file_path).exists():
            print(f"❌ Error: File not found - {file_path}")
            return
    print("📥 Loading result files...")
    results_dict = load_and_process_files(CMC_FILE, SKATO_FILE, VT_FILE)
    if results_dict is None:
        print("❌ Failed to load one or more input files. Exiting.")
        return
    # Process input datasets
    for test_type, df in results_dict.items():
        results_dict[test_type] = calculate_significance_metrics(df)
    print("\n🔄 Merging results...")
    combined_df = merge_results(results_dict)
    print("⭐ Selecting best result per gene...")
    best_results = find_best_result_per_gene(combined_df)
    print("🔗 Analysing cross-test significance...")
    cross_test_df, all_tests_sig, two_tests_sig, one_test_sig = analyze_cross_test_significance(combined_df)
    # Console summary
    generate_summary_report(results_dict, combined_df, best_results, cross_test_df,
                            all_tests_sig, two_tests_sig, one_test_sig)
    # Apply multiple testing correction to each test type
    print("Applying multiple testing corrections...")
    for test_type, df in results_dict.items():
        n_tests = len(df)
        thresholds = calculate_significance_thresholds(n_tests)
        results_dict[test_type] = apply_multiple_testing_correction(df)
        # Summary of significant results
        print(f"\n{test_type} Results Summary:")
        print(f"  Total tests: {n_tests}")
        print(f"  Nominal sig (p<0.05): {(df['P_value'] < 0.05).sum()}")
        print(f"  Bonferroni sig: {(df['p_bonferroni'] < 0.05).sum() if 'p_bonferroni' in df else 'N/A'}")
        print(f"  FDR sig: {(df['sig_fdr_bh']).sum() if 'sig_fdr_bh' in df else 'N/A'}")
    # For gene-level analysis (best result per gene)
    print("\nApplying gene-level multiple testing correction...")
    n_genes = len(best_results)
    gene_thresholds = calculate_significance_thresholds(n_genes)
    best_results_corrected = apply_multiple_testing_correction(best_results)
    display_cols = ['Gene', 'Test_Type', 'P_value', 'p_fdr_bh', 'NumVar', 'NumPolyVar']
    print(f"\nGene-level Results Summary:")
    print(f"  Total genes: {n_genes}")
    print(f"  Nominal sig (p<0.05): {(best_results['P_value'] < 0.05).sum()}")
    print(f"  Bonferroni sig: {(best_results_corrected['p_bonferroni'] < 0.05).sum()}")
    print(f"  FDR sig: {(best_results_corrected['sig_fdr_bh']).sum()}")
    # Update your reporting to include corrected results
    print(f"\nTop significant genes (FDR corrected):")
    fdr_significant = best_results_corrected[best_results_corrected['sig_fdr_bh']]
    if len(fdr_significant) > 0:
        print(fdr_significant[display_cols].head(10).to_string(index=False, float_format='%.2e'))
    else:
        print("No genes significant after FDR correction")
        # Show top Bonferroni significant if any
        bonf_significant = best_results_corrected[best_results_corrected['sig_bonferroni']]
        if len(bonf_significant) > 0:
            print(f"\nTop significant genes (Bonferroni corrected):")
            print(bonf_significant[display_cols].head(5).to_string(index=False, float_format='%.2e'))
    # Save corrected results
    best_results_corrected.to_csv(OUTPUT_DIR / "best_results_per_gene_corrected.csv", index=False)
    # Additional reporting of multiple testing correction stats and lists
    generate_correction_summary_report(best_results_corrected, OUTPUT_DIR)
    # Save
    output_path = save_results(combined_df, best_results, cross_test_df, OUTPUT_DIR)
    print("\n🏁 Analysis complete.")
    print(f"- {len(all_tests_sig)} genes significant in ALL tests")
    print(f"- {len(two_tests_sig)} genes significant in 2 tests")
    print(f"- {len(one_test_sig)} genes significant in 1 test")
    print(f"\n📌 Priority genes (>=2 tests significant):")
    priority_genes = cross_test_df[cross_test_df['Tests_Significant'] >= 2]['Gene'].tolist()
    if priority_genes:
        for i, gene in enumerate(priority_genes[:MAX_GENES_TO_LIST], 1):
            gene_info = cross_test_df[cross_test_df['Gene'] == gene].iloc[0]
            print(f"{i}. {gene} ({gene_info['Tests_Significant']} tests, min p = {gene_info['Min_Pvalue']:.2e})")
        if len(priority_genes) > MAX_GENES_TO_LIST:
            print(f"... and {len(priority_genes) - MAX_GENES_TO_LIST} more genes.")
    else:
        print("No multi-test significant genes detected.")
if __name__ == "__main__":
    main()

### 05 Variant Extraction
- Extract significant loci from VCF using bcftools
- Apply allele frequency filters for rare variant validation
- Generate RSID lists for candidate genes




In [ ]:
# example of variant extraction premise

# from Rvtests results
# find variants in CDCA7L 7:21907041-21952042

bcftools view -r 7:21907041-21952042 "/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_annotations/inputs/vcf/ADNI2_AD_CN_converted.vcf.gz" \
  | bcftools view -i 'AF >0.005 && AF <=0.01' \
  > "/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/04_post_analysis/CDCA7L_ADNI_rare.vcf"


In [ ]:
# 05_post_analysis
# i. extract variants from significant regions with bcftools


##############################################################
# USER-DEFINED INPUTS (EDIT HERE) #
##############################################################

# Input files (comment out unused to select input)
SIGNIFICANT_GENES_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/04_post_analysis/multi_test_significant_genes.csv"
# SIGNIFICANT_GENES_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/04_post_analysis/significant_results.csv"
# SIGNIFICANT_GENES_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/04_post_analysis/best_results_per_gene.csv"

# Input VCF file
VCF_FILE="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/02_dataset_conversion/ADNI2_AD_CN_converted.vcf.gz"

# Output directory
OUTPUT_DIR="/mnt/c/Users/B00731414/OneDrive - Ulster University/6. Code/ADNI/5. Rvtests/05_variant_extraction"

# Allele frequency filter (rare variants)
MIN_AF="0.005"
MAX_AF="0.01"

# Additional bcftools filters
ADDITIONAL_FILTERS=""   # e.g. "&& QUAL>20 && DP>10"

# Processing options
MAX_GENES_TO_PROCESS="20"   # Leave empty to run all genes
SKIP_EXISTING_FILES="true"  # Skip existing files if true, overwrite if false

##############################################################
# DO NOT EDIT BELOW THIS LINE #
##############################################################

# ── Console Colours ─────────────────────────────
RED='\033[0;31m'
GREEN='\033[0;32m'
YELLOW='\033[1;33m'
BLUE='\033[0;34m'
NC='\033[0m' # No Colour


# ── Dependency check ─────────────────────────────
check_dependencies() {
    echo -e "${BLUE}🔍 Checking dependencies...${NC}"
    if ! command -v bcftools &>/dev/null; then
        echo -e "${RED}❌ Error: bcftools not found. Please install bcftools.${NC}"
        exit 1
    fi
    if ! command -v awk &>/dev/null; then
        echo -e "${RED}❌ Error: awk not found. Please install awk.${NC}"
        exit 1
    fi
    echo -e "${GREEN}✅ All dependencies present.${NC}"
}


# ── Parse genomic range ─────────────────────────────
parse_genomic_range() {
    local range_string="$1"
    range_string=$(echo "$range_string" | sed 's/"//g' | cut -d',' -f1)
    if [[ $range_string =~ ^([^:]+):([0-9]+)-([0-9]+)$ ]]; then
        CHROM="${BASH_REMATCH[1]}"
        START="${BASH_REMATCH[2]}"
        END="${BASH_REMATCH[3]}"
        return 0
    else
        return 1
    fi
}


# ── Extract variants for a gene ─────────────────────────────
extract_variants_for_gene() {
    local gene_name="$1"
    local chrom="$2"
    local start="$3"
    local end="$4"

    local output_file="${OUTPUT_DIR}/${gene_name}_ADNI_rare.vcf"
    local rsid_file="${OUTPUT_DIR}/${gene_name}_ADNI_rare_rsids.txt"

    # Skip pre-existing files
    if [[ "$SKIP_EXISTING_FILES" == "true" && -f "$output_file" ]]; then
        echo -e "${YELLOW}⚠️  Skipping $gene_name - file already exists${NC}"
        return 0
    fi

    local region="${chrom}:${start}-${end}"
    local af_filter="AF >${MIN_AF} && AF <=${MAX_AF}"
    local full_filter="$af_filter ${ADDITIONAL_FILTERS}"

    echo -e "${BLUE}📂 Extracting: $gene_name ($region)...${NC}"

    # Run bcftools
    if bcftools view -r "$region" "$VCF_FILE" | bcftools view -i "$full_filter" > "$output_file" 2>/dev/null; then
        if [[ -f "$output_file" ]]; then
            local file_size=$(stat -c%s "$output_file" 2>/dev/null || echo "0")
            local variant_count=$(grep -v '^#' "$output_file" 2>/dev/null | wc -l || echo "0")

            if [[ $variant_count -gt 0 ]]; then
                echo -e "${GREEN}✅ Success: $variant_count variants extracted (${file_size} bytes).${NC}"
                echo "$gene_name,Success,$variant_count variants (${file_size} bytes),${gene_name}_ADNI_rare.vcf,$chrom,$start,$end" >> "$results_file"

                # 🔹 Extract RSIDs into a separate txt file
                awk '!/^#/ {print $3}' "$output_file" | grep -v '^\.$' > "$rsid_file"
                rsid_count=$(wc -l < "$rsid_file")
                echo -e "${GREEN}📄 RSID list created: $rsid_file (${rsid_count} IDs)${NC}"

            else
                echo -e "${YELLOW}ℹ️  Success: No variants matched AF filter.${NC}"
                echo "$gene_name,Success,No variants found,${gene_name}_ADNI_rare.vcf,$chrom,$start,$end" >> "$results_file"
            fi

        else
            echo -e "${RED}❌ Failed: Output file not created.${NC}"
            echo "$gene_name,Failed,Output file not created,N/A,$chrom,$start,$end" >> "$results_file"
        fi
    else
        echo -e "${RED}❌ Failed: bcftools command failed.${NC}"
        echo "$gene_name,Failed,bcftools error,N/A,$chrom,$start,$end" >> "$results_file"
    fi
}


# ── Main ─────────────────────────────
main() {
    echo "================================================"
    echo "GENE VARIANT EXTRACTION"
    echo "================================================"
    echo "Input genes: $SIGNIFICANT_GENES_FILE"
    echo "VCF file:    $VCF_FILE"
    echo "Output dir:  $OUTPUT_DIR"
    echo "AF filter:   $MIN_AF < AF <= $MAX_AF"
    [[ -n "$ADDITIONAL_FILTERS" ]] && echo "Extra filters: $ADDITIONAL_FILTERS"
    [[ -n "$MAX_GENES_TO_PROCESS" ]] && echo "Gene cap: $MAX_GENES_TO_PROCESS"
    echo "Skip existing files: $SKIP_EXISTING_FILES"
    echo "================================================"

    # Check tools
    check_dependencies

    # Validate files
    [[ ! -f "$SIGNIFICANT_GENES_FILE" ]] && echo -e "${RED}❌ Error: Input CSV missing.${NC}" && exit 1
    [[ ! -f "$VCF_FILE" ]] && echo -e "${RED}❌ Error: VCF missing.${NC}" && exit 1

    mkdir -p "$OUTPUT_DIR"

    # Log file
    results_file="$OUTPUT_DIR/variant_extraction_results.csv"
    echo "Gene,Status,Message,Output_File,Chromosome,Start,End" > "$results_file"

    # Identify columns
    header_line=$(head -n 1 "$SIGNIFICANT_GENES_FILE")
    echo "📑 CSV header: $header_line"

    gene_col=$(echo "$header_line" | tr ',' '\n' | grep -n "Gene" | cut -d':' -f1)
    range_col=$(echo "$header_line" | tr ',' '\n' | grep -n "RANGE" | cut -d':' -f1)
    [[ -z "$gene_col" || -z "$range_col" ]] && echo -e "${RED}❌ Gene/RANGE columns not found.${NC}" && exit 1
    echo "✅ Found Gene @ col $gene_col, RANGE @ col $range_col"

    local processed_count=0

    # Process CSV
    tail -n +2 "$SIGNIFICANT_GENES_FILE" | while IFS= read -r line; do
        [[ -n "$MAX_GENES_TO_PROCESS" && $processed_count -ge $MAX_GENES_TO_PROCESS ]] && break
        IFS=',' read -ra FIELDS <<< "$line"
        local gene_name=$(echo "${FIELDS[$((gene_col-1))]}" | sed 's/"//g')
        local range_string=$(echo "${FIELDS[$((range_col-1))]}" | sed 's/"//g')

        echo -e "\n🔬 Processing gene $((processed_count+1)): $gene_name"
        echo "   Range: $range_string"

        if parse_genomic_range "$range_string"; then
            echo "   Parsed: $CHROM:$START-$END"
            extract_variants_for_gene "$gene_name" "$CHROM" "$START" "$END"
        else
            echo -e "${RED}❌ Failed: Invalid genomic range '${range_string}'${NC}"
            echo "$gene_name,Failed,Invalid range,N/A,N/A,N/A,N/A" >> "$results_file"
        fi
        ((processed_count++))
    done

    echo -e "\n🏁 Pipeline complete."
}


# ── Run ─────────────────────────────
main "$@"
